In [11]:
import os
import pandas as pd
import numpy as np

In [12]:
PROCESSED_DIR = "../data/processed"
INPUT_CSV = os.path.join(PROCESSED_DIR, "corridor_shipments_clean.csv")

In [13]:
# Outputs from this notebook
FEATURES_CSV = os.path.join(PROCESSED_DIR, "corridor_features.csv")
FEATURES_PARQUET = os.path.join(PROCESSED_DIR, "corridor_features.parquet")

os.makedirs(PROCESSED_DIR, exist_ok=True)

#### Load Cleaned Data

In [14]:
# -----------------------------
# Load the cleaned dataset (Notebook 01 output)
# -----------------------------
date_cols = ["depart_time", "border_arrival_time", "border_exit_time", "arrival_time"]

df = pd.read_csv(INPUT_CSV, parse_dates=date_cols)

print("Loaded:", df.shape)
df.head()

Loaded: (1000, 19)


,shipment_id,truck_id,origin_port,border,destination,depart_time,border_arrival_time,border_exit_time,arrival_time,goods_category,weight_kg,declared_value_usd,fuel_price_usd_per_litre,rainfall_indicator,delay_reason,travel_to_border_hours,clearance_hours,travel_to_destination_hours,total_transit_hours
0,SHP00001,TRK754,Mombasa,Malaba,Kampala,2024-01-29 00:00:00,2024-01-29 12:26:57.490795,2024-01-29 19:37:49.228206,2024-01-30 00:36:35.670901,Electronics,16047.07,20438.99,1.55,0,NaN,12.449303,7.181038,4.979567,24.609909
1,SHP00002,TRK704,Mombasa,Malaba,Kampala,2024-04-18 01:00:00,2024-04-18 09:10:43.619940,2024-04-18 13:39:43.839026,2024-04-18 18:32:12.225863,Construction Materials,27040.37,32255.69,1.46,1,NaN,8.178783,4.483394,4.874552,17.536729
2,SHP00003,TRK325,Mombasa,Malaba,Kampala,2024-04-24 18:00:00,2024-04-25 03:40:08.919298,2024-04-25 07:48:57.582009,2024-04-25 15:17:35.506628,Textiles,10252.37,13212.20,1.27,0,NaN,9.669144,4.146851,7.477201,21.293196
3,SHP00004,TRK320,Mombasa,Malaba,Kampala,2024-03-27 03:00:00,2024-03-27 11:33:23.310217,2024-03-27 15:42:12.174996,2024-03-27 20:05:24.890822,Textiles,6095.60,6005.17,1.12,1,NaN,8.556475,4.146907,4.386866,17.090247
4,SHP00005,TRK649,Mombasa,Malaba,Kampala,2024-02-01 12:00:00,2024-02-01 20:28:22.084278,2024-02-02 10:25:18.999930,2024-02-02 15:35:40.767182,Pharmaceuticals,5711.12,8013.81,1.39,0,Customs Inspection,8.472801,13.949143,5.172713,27.594658


#### Core Time Features

In [15]:
# -----------------------------
# Core duration features (hours)
# -----------------------------
# These are the main operational metrics for corridor efficiency.
df["travel_to_border_hours"] = (
    df["border_arrival_time"] - df["depart_time"]
).dt.total_seconds() / 3600

df["clearance_hours"] = (
    df["border_exit_time"] - df["border_arrival_time"]
).dt.total_seconds() / 3600

df["travel_to_destination_hours"] = (
    df["arrival_time"] - df["border_exit_time"]
).dt.total_seconds() / 3600

df["total_transit_hours"] = (
    df["arrival_time"] - df["depart_time"]
).dt.total_seconds() / 3600

# Delay label (binary) for classification tasks
df["is_delayed"] = (df["clearance_hours"] > 12).astype(int)

df[["shipment_id", "travel_to_border_hours", "clearance_hours", "total_transit_hours", "is_delayed"]].head()

,shipment_id,travel_to_border_hours,clearance_hours,total_transit_hours,is_delayed
0,SHP00001,12.449303,7.181038,24.609909,0
1,SHP00002,8.178783,4.483394,17.536729,0
2,SHP00003,9.669144,4.146851,21.293196,0
3,SHP00004,8.556475,4.146907,17.090247,0
4,SHP00005,8.472801,13.949143,27.594658,1


#### Calendar & Seasonality Features

In [16]:
# -----------------------------
# Calendar features for patterns + seasonality
# -----------------------------
df["depart_date"] = df["depart_time"].dt.date
df["depart_year"] = df["depart_time"].dt.year
df["depart_month"] = df["depart_time"].dt.month
df["depart_day"] = df["depart_time"].dt.day

# Day-of-week features
df["depart_day_of_week_num"] = df["depart_time"].dt.dayofweek  # Monday=0 ... Sunday=6
df["depart_day_name"] = df["depart_time"].dt.day_name()

# Hour-of-day features (useful if congestion depends on time)
df["depart_hour"] = df["depart_time"].dt.hour

# Weekend flag
df["is_weekend"] = df["depart_day_of_week_num"].isin([5, 6]).astype(int)

df[["shipment_id", "depart_time", "depart_day_name", "depart_hour", "is_weekend"]].head()

,shipment_id,depart_time,depart_day_name,depart_hour,is_weekend
0,SHP00001,2024-01-29 00:00:00,Monday,0,0
1,SHP00002,2024-04-18 01:00:00,Thursday,1,0
2,SHP00003,2024-04-24 18:00:00,Wednesday,18,0
3,SHP00004,2024-03-27 03:00:00,Wednesday,3,0
4,SHP00005,2024-02-01 12:00:00,Thursday,12,0


#### Operational Flags (Bottlenecks & Business Logic)

In [17]:
# -----------------------------
# Operational flags: help segmentation and storytelling
# -----------------------------

# "Long clearance" buckets (makes dashboards cleaner)
# You can tune these thresholds based on what you want to communicate.
df["clearance_bucket"] = pd.cut(
    df["clearance_hours"],
    bins=[-0.01, 2, 6, 12, 24, np.inf],
    labels=["0–2h", "2–6h", "6–12h", "12–24h", "24h+"]
)

# "Long transit" bucket
df["transit_bucket"] = pd.cut(
    df["total_transit_hours"],
    bins=[-0.01, 12, 18, 24, 36, np.inf],
    labels=["0–12h", "12–18h", "18–24h", "24–36h", "36h+"]
)

# High value shipment flag (top 10% declared value)
p90_value = df["declared_value_usd"].quantile(0.90)
df["is_high_value"] = (df["declared_value_usd"] >= p90_value).astype(int)

# Heavy shipment flag (top 10% weight)
p90_weight = df["weight_kg"].quantile(0.90)
df["is_heavy"] = (df["weight_kg"] >= p90_weight).astype(int)

df[["shipment_id", "clearance_bucket", "transit_bucket", "is_high_value", "is_heavy"]].head()

,shipment_id,clearance_bucket,transit_bucket,is_high_value,is_heavy
0,SHP00001,6–12h,24–36h,0,0
1,SHP00002,2–6h,12–18h,0,0
2,SHP00003,2–6h,18–24h,0,0
3,SHP00004,2–6h,12–18h,0,0
4,SHP00005,12–24h,24–36h,0,0


#### Rolling & Trend Features (Time-Aware)

In [18]:
# -----------------------------
# Time-aware rolling metrics
# -----------------------------
# Sort by depart_time to compute rolling features correctly.
df = df.sort_values("depart_time").reset_index(drop=True)

# Rolling averages for corridor congestion signals
# (window=30 means last 30 shipments; you can also do daily rolling later)
df["rolling_clearance_mean_30"] = df["clearance_hours"].rolling(window=30, min_periods=10).mean()
df["rolling_delay_rate_30"] = df["is_delayed"].rolling(window=30, min_periods=10).mean()

# Rolling median is robust to outliers
df["rolling_clearance_median_30"] = df["clearance_hours"].rolling(window=30, min_periods=10).median()

df[["depart_time", "clearance_hours", "rolling_clearance_mean_30", "rolling_delay_rate_30"]].head(15)

,depart_time,clearance_hours,rolling_clearance_mean_30,rolling_delay_rate_30
0,2024-01-01 03:00:00,8.522000,NaN,NaN
1,2024-01-01 12:00:00,4.780147,NaN,NaN
2,2024-01-01 16:00:00,20.000000,NaN,NaN
3,2024-01-01 19:00:00,11.657312,NaN,NaN
4,2024-01-02 00:00:00,12.444743,NaN,NaN
5,2024-01-02 08:00:00,9.863236,NaN,NaN
6,2024-01-02 16:00:00,9.585429,NaN,NaN
7,2024-01-02 17:00:00,17.981120,NaN,NaN
8,2024-01-02 22:00:00,10.365941,NaN,NaN
9,2024-01-03 02:00:00,7.084482,11.228441,0.300000


#### Encode Delay Reason (Analytics-Friendly)

In [19]:
# -----------------------------
# Normalize delay_reason into groups (optional)
# -----------------------------
# This helps dashboards and modeling (fewer unique categories).
reason_map = {
    "None": "None",
    "Customs Inspection": "Border/Customs",
    "Documentation Issues": "Border/Customs",
    "System Downtime": "Border Systems",
    "Traffic Congestion": "Road/Traffic",
    "Weather Disruption": "Weather",
    "Truck Breakdown": "Vehicle/Mechanical"
}

df["delay_reason_group"] = df["delay_reason"].map(reason_map).fillna("Other")

df[["delay_reason", "delay_reason_group"]].value_counts().head(10)

delay_reason          delay_reason_group
Customs Inspection    Border/Customs        23
Weather Disruption    Weather               20
Truck Breakdown       Vehicle/Mechanical    20
Traffic Congestion    Road/Traffic          19
Documentation Issues  Border/Customs        13
System Downtime       Border Systems        12
Name: count, dtype: int64

#### Cost Features (for impact analysis)

In [20]:
# -----------------------------
# Cost-of-delay simulation feature (for dashboards + reports)
# -----------------------------
# Use a simple assumption; you can parameterize it later.
COST_PER_HOUR_USD = 150

# Only charge cost when delayed (clearance > 12 hours)
df["delay_cost_usd"] = np.where(
    df["is_delayed"] == 1,
    df["clearance_hours"] * COST_PER_HOUR_USD,
    0
)

df[["shipment_id", "clearance_hours", "is_delayed", "delay_cost_usd"]].head()

,shipment_id,clearance_hours,is_delayed,delay_cost_usd
0,SHP00823,8.522000,0,0.000000
1,SHP00640,4.780147,0,0.000000
2,SHP00639,20.000000,1,3000.000000
3,SHP00842,11.657312,0,0.000000
4,SHP00652,12.444743,1,1866.711417


#### Model-Ready” Feature Table

In [21]:
# -----------------------------
# Select columns for modeling
# -----------------------------
feature_cols = [
    # identifiers (keep for joins / tracing; exclude from ML training if needed)
    "shipment_id", "truck_id",

    # numeric drivers
    "weight_kg", "declared_value_usd", "fuel_price_usd_per_litre", "rainfall_indicator",
    "travel_to_border_hours", "clearance_hours", "travel_to_destination_hours", "total_transit_hours",

    # calendar drivers
    "depart_month", "depart_day_of_week_num", "depart_hour", "is_weekend",

    # operational flags
    "is_high_value", "is_heavy",

    # rolling congestion signals
    "rolling_clearance_mean_30", "rolling_clearance_median_30", "rolling_delay_rate_30",

    # categories for analytics (for dashboards or encoding later)
    "goods_category", "delay_reason_group"
]

target_col = "is_delayed"

features_df = df[feature_cols + [target_col]].copy()

print("Feature table shape:", features_df.shape)
features_df.head()

Feature table shape: (1000, 22)


,shipment_id,truck_id,weight_kg,declared_value_usd,fuel_price_usd_per_litre,rainfall_indicator,travel_to_border_hours,clearance_hours,travel_to_destination_hours,total_transit_hours,...,depart_hour,is_weekend,is_high_value,is_heavy,rolling_clearance_mean_30,rolling_clearance_median_30,rolling_delay_rate_30,goods_category,delay_reason_group,is_delayed
0,SHP00823,TRK600,28351.50,34054.16,1.38,1,11.285224,8.522000,4.270258,24.077482,...,3,0,0,0,NaN,NaN,NaN,Fuel,Other,0
1,SHP00640,TRK331,10666.48,12530.05,1.60,1,13.724932,4.780147,7.271907,25.776986,...,12,0,0,0,NaN,NaN,NaN,Machinery,Other,0
2,SHP00639,TRK688,17593.27,24469.33,1.49,0,12.245634,20.000000,7.083963,39.329596,...,16,0,0,0,NaN,NaN,NaN,Machinery,Weather,1
3,SHP00842,TRK761,3041.22,3616.17,1.45,1,10.859174,11.657312,7.605780,30.122266,...,19,0,0,0,NaN,NaN,NaN,Pharmaceuticals,Other,0
4,SHP00652,TRK182,5562.38,5554.66,1.45,1,9.076477,12.444743,4.356565,25.877785,...,0,0,0,0,NaN,NaN,NaN,Pharmaceuticals,Vehicle/Mechanical,1


#### Quick Feature Quality Checks

In [22]:
# -----------------------------
# Quick checks: missingness + weird values
# -----------------------------
missing = features_df.isnull().mean().sort_values(ascending=False)
print("Top missingness rates:")
display(missing.head(10))

# If rolling features have missing values early (expected), you can fill them.
# Example strategy:
features_df["rolling_clearance_mean_30"] = features_df["rolling_clearance_mean_30"].fillna(features_df["clearance_hours"].mean())
features_df["rolling_clearance_median_30"] = features_df["rolling_clearance_median_30"].fillna(features_df["clearance_hours"].median())
features_df["rolling_delay_rate_30"] = features_df["rolling_delay_rate_30"].fillna(features_df["is_delayed"].mean())

# Verify no missing values in key numeric features after fill
key_numeric = [
    "weight_kg", "declared_value_usd", "fuel_price_usd_per_litre", "rainfall_indicator",
    "travel_to_border_hours", "clearance_hours", "total_transit_hours",
    "rolling_clearance_mean_30", "rolling_delay_rate_30"
]
print("Missing after fill (key features):")
display(features_df[key_numeric].isnull().sum())

Top missingness rates:


rolling_delay_rate_30          0.009
rolling_clearance_mean_30      0.009
rolling_clearance_median_30    0.009
weight_kg                      0.000
shipment_id                    0.000
truck_id                       0.000
rainfall_indicator             0.000
fuel_price_usd_per_litre       0.000
declared_value_usd             0.000
travel_to_border_hours         0.000
dtype: float64

Missing after fill (key features):


weight_kg                    0
declared_value_usd           0
fuel_price_usd_per_litre     0
rainfall_indicator           0
travel_to_border_hours       0
clearance_hours              0
total_transit_hours          0
rolling_clearance_mean_30    0
rolling_delay_rate_30        0
dtype: int64

#### Save Feature Dataset

In [23]:
# -----------------------------
# Save feature dataset to processed folder
# -----------------------------
features_df.to_csv(FEATURES_CSV, index=False)
print("Saved:", FEATURES_CSV)

# Parquet is faster & preserves types well (recommended)
features_df.to_parquet(FEATURES_PARQUET, index=False)
print("Saved:", FEATURES_PARQUET)

Saved: ../data/processed\corridor_features.csv
Saved: ../data/processed\corridor_features.parquet
